In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
# %matplotlib qt
# Plot in external window

In [ ]:
from libs.common import get_images_in_dir, get_files_in_dir, plot_images, put_data
from libs.calibration import CameraCalibrator
from libs.filter_image import filter_image
from libs.find_lanes import fit_polynomial, get_road, find_new_fit, get_both_fitx

In [ ]:
test_images_dir = 'test_images/'
test_images = get_images_in_dir(test_images_dir)
image_names = list(get_files_in_dir(test_images_dir))

## Calibrate camera

### Calibrate

In [ ]:
calibration_data_dir = 'camera_cal/'

calibrator = CameraCalibrator(9, 6)

calibrate_images = get_images_in_dir(calibration_data_dir)
used_images = []

for image in calibrate_images:
    if calibrator.calibrate(image):
        used_images.append(image)


calibrator.finish_calibrate()

## Find lanes

In [ ]:
def put_curvature_dist(image, curvature, dist):
    cv2.putText(image, 'Curvature is {:0.0f}m'.format(curvature), (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255,255), thickness=3)
    cv2.putText(image, 'Offset is {0:0.2f}m from centre'.format(dist), (100, 160), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255,255), thickness=3)

In [ ]:
left = [5.619491003016991e-06, -0.022389297191894888, 207.78001585426344] 
right = [-0.00017266842079864529, 0.13416494883492175, 1101.124026129575]
def find_lanes(image):
    left_fit, right_fit = find_new_fit(image, left, right)
    return get_both_fitx(left_fit, right_fit, image.shape)


In [ ]:

fig = plt.figure(figsize=[25, 5 * len(image_names)])
for image_id, file_name in enumerate(image_names):
    image = mpimg.imread(file_name)
    undistored = calibrator.undistort(image)
    transform = calibrator.transform(undistored)
    filtered = filter_image(transform)    
    road_bv, data = get_road(filtered, find_lanes)
    road = calibrator.un_transform(road_bv)
    
    res = cv2.addWeighted(undistored, 1, road, 0.3, 0)
    put_data(res, data)
    result = [image, res]  
    axes = plot_images(fig, result, image_id, len(test_images))
    axes[0].set_title(file_name)
    